In [16]:
import os
from os import walk
import SimpleITK as sitk
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import matplotlib.colors as colors
from matplotlib import cm
import numpy as np
import pandas as pd
from medpy.io import load, save
import random
import PIL
from PIL import Image
import scipy.ndimage
import json
import math
from tqdm import tqdm
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20, 12)

In [4]:
config = json.load(open('config.json'))
data_folder_path = config["data_folder_path"]

ct_folder = f'{data_folder_path}/luna16_nodules/nodule_patches/'
mask_folder = f'{data_folder_path}/luna16_nodules/segmentation/'
original_xray_folder = f"{data_folder_path}/original/"
xray_folder = f"{data_folder_path}/preprocessed/"
lung_segment_folder = f"{data_folder_path}/generation_data/lung_segmentations/"

In [5]:
def resample_nodule(ct_data, xray_spacing):
    resize_factor = [1,1,1]/xray_spacing
    new_real_shape = ct_data.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / ct_data.shape
    new_spacing = [1,1,1] / real_resize_factor
    resampled_nodule = scipy.ndimage.interpolation.zoom(ct_data, real_resize_factor, mode='nearest')
    return resampled_nodule

def segment_nodule(ct, mask):
    background = np.full(ct.shape, np.min(ct))
    segmented_nodule = np.where(mask == 0, background, ct)
    return segmented_nodule

def crop_nodule(ct_2d):
    outlines = np.where(ct_2d != np.min(ct_2d))
    x_min = np.min(outlines[0])
    x_max = np.max(outlines[0])
    y_min = np.min(outlines[1])
    y_max = np.max(outlines[1])

    cropped_patch = ct_2d[x_min:x_max+1, y_min:y_max+1]

    return cropped_patch

#Code from the baseline shared by Ecem
def project_ct(X_ct, p_lambda = 0.85):
    '''
    Generate 2D digitally reconstructed radiographs from CT scan. (DRR, fake CXR, simulated CXR)
    X_ct: CT scan
    p-lambda:  β controls the boosting of X-ray absorption as the tissue density increases.
    We have chosen β=0.85 for our experiments after performing a visual comparison with real chest X-rays.
    author: Ecem Sogancioglu
    '''
    X_ct[X_ct > 400] = 400
    X_ct[X_ct < -500] = -500
    X_ct[X_ct < -1024] = -1024
    X_ct += 1024
    # 1424 524 698.748232
    X_ct = X_ct/1000.0
    X_ct *= p_lambda
    X_ct[X_ct > 1] = 1
    #1.0 0.4454 0.5866707652
    X_ct_2d = np.mean(np.exp(X_ct), axis=1)
    return X_ct_2d

def preprocess_nodule(xray_header, ct, mask):
    #Resample the ct and the mask based on Xray spacing

    xray_spacing = xray_header.get_voxel_spacing()
    resampled_ct = resample_nodule(ct, np.array([xray_spacing[0],xray_spacing[1],xray_spacing[0]]))
    resampled_mask = resample_nodule(mask, np.array([xray_spacing[0],xray_spacing[1],xray_spacing[0]]))

    #Segment the nodule using the mask
    segmented_ct = segment_nodule(resampled_ct, resampled_mask)

    #Project the segmented CT data from 3d to 2d
    ct_2d = project_ct(segmented_ct)

    return ct_2d

In [25]:
from multiprocessing.pool import ThreadPool as Pool

def nodule_check_func(i):
    try:
        ct = load(os.path.join(ct_folder, ct_names[i]))[0]
        mask = load(os.path.join(mask_folder, mask_names[i]))[0]
        xray, xray_header = load(os.path.join(xray_folder, xray_names[i]))

        nodule_2d = preprocess_nodule(xray_header=xray_header,
                                   mask=mask,
                                   ct=ct)
        cropped_nodule = crop_nodule(nodule_2d)
        if cropped_nodule.shape[0] <= 30 and cropped_nodule.shape[1] <= 30:
            save(cropped_nodule, f"{data_folder_path}/valid_nodules/{i}.mha")
    except:
        print("Something went wrong")

_, _, ct_names = next(walk(f'{data_folder_path}/luna16_nodules/nodule_patches'))
_, _, mask_names = next(walk(f'{data_folder_path}/luna16_nodules/segmentation'))
_, _, xray_names = next(walk(xray_folder))

pool = Pool(10)

for i in range(len(ct_names)):
    pool.apply_async(nodule_check_func, (i,))

pool.close()
pool.join()

Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something wen